EDAF75 - notebook for lab 2
===========================

**Author:** Christian Söderberg



In [1]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

## EDAF75 - lab 2: Experimenting with your own database



To begin, we need to enable SQL in our notebook:



In [2]:
!sqlite3 movies.sqlite < lab2.sql

In [3]:
%load_ext sql

If we use `jupysql` (instead of `ipython-sql`) we get a default limit of 10 rows for any query &#x2013; to turn this limit off we can write:



In [4]:
%config SqlMagic.displaylimit = None

We can now load the lab database:



In [5]:
%sql sqlite:///movies.sqlite

'Connected: @movies.sqlite'

We want to make sure that SQLite3 really checks our foreign
key constraints &#x2013; to do that, we run:



In [6]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:

-   Show the names of all movies.



In [7]:
%%sql
SELECT  title
FROM    movies

 * sqlite:///movies.sqlite
Done.


title
Blade Runner 2049
Harakiri
Casablanca
Patlabor: The Movie
Citizen Kane


-   Show the performance dates for one of the movies.



In [8]:
%%sql
SELECT  date
FROM    screening
WHERE   imdb_key = 'tt0056058'

 * sqlite:///movies.sqlite
Done.


date
2025-03-01
2025-03-03
2025-03-04


-   Show all data concerning performances at a given theatere on a given date.



In [9]:
%%sql
SELECT  *
FROM    screening
WHERE   date = '2025-03-03' AND  theatre = 'Filmstaden Malmö'

 * sqlite:///movies.sqlite
Done.


id,start_time,theatre,date,imdb_key
31250d560e181ee4ce8cf7370ff29c9b,20:30,Filmstaden Malmö,2025-03-03,tt0034583


-   List all customers



In [10]:
%%sql
SELECT      full_name
FROM        customers
ORDER BY    full_name 

 * sqlite:///movies.sqlite
Done.


full_name
Erik Gustavsson
Ludwig Lundstedt
Mikolaj Sinicka


-   List all tickets



In [11]:
%%sql
SELECT  *
FROM    tickets

 * sqlite:///movies.sqlite
Done.


id,username,date_and_time


-   Create a new ticket to some performance (i.e., insert a new row in your table of tickets).



In [19]:
%%sql
INSERT
INTO    tickets
VALUES ((lower(hex(randomblob(16)))), 'miko', '31250d560e181ee4ce8cf7370ff29c9b')

 * sqlite:///movies.sqlite
1 rows affected.


[]

In [20]:
%%sql
SELECT  *
FROM    tickets

 * sqlite:///movies.sqlite
Done.


id,username,date_and_time
35f143311577dc479e284df1a40b074c,miko,31250d560e181ee4ce8cf7370ff29c9b


In newer versions of SQLite (since version 3.35, released in March 2021), and in [PostgreSQL]([https://www.postgresql.org/docs/current/sql-insert.html](https://www.postgresql.org/docs/current/sql-insert.html)), we can get any value generated during an insert using the `INSERT...-RETURNING` statement:

<pre>
<code>INSERT</code>
<code>INTO       students</code>
<code>VALUES     ('Amy', 3.9, 1200)</code>
<code>RETURNING  s&#95;id</code>
</pre>

which would return the generated `s_id` for the new student.

If your SQLite version is older than 3.35, and you can't upgrade, you can instead use the following idea: each row in a SQLite3 table has a `rowid` attribute, it is a unique integer which essentially tells in which order the rows were inserted, and it's not displayed in queries unless we ask for it. SQLite3 also have a function, `last_insert_rowid()`, which returns the `rowid` of the last inserted row of a table, so we can see the `s_id` of the most recently inserted student with the following query:

<pre>
<code>SELECT s&#95;id</code>
<code>FROM   students</code>
<code>WHERE  rowid = last&#95;insert&#95;rowid();</code>
</pre>

Now, check what ticket number we got for the ticket we created above (it should be the same as the ticket id, which should be a `randomblob`):



In [14]:
%%sql
SELECT  id
FROM    tickets
WHERE   rowid = last_insert_rowid();

 * sqlite:///movies.sqlite
Done.


id


-   Try to insert two movie theaters with the same name (this
    should fail).



In [15]:
%%sql
INSERT
INTO    theaters
VALUES  ('Kino', 100)

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theaters.name
[SQL: INSERT
INTO    theaters
VALUES  ('Kino', 100)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

-   Try to insert a performance where the theater doesn’t exist in the database (this should fail).



In [16]:
%%sql
INSERT
INTO    screening
VALUES (lower(hex(randomblob(16))), '18:30', 'Kino', '2025-03-03', 'tt0056058')

 * sqlite:///movies.sqlite
1 rows affected.


[]

In [17]:
%%sql
INSERT
INTO    screening
VALUES (lower(hex(randomblob(16))), '18:30', 'Södran', '2025-03-03', 'tt0056058')

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT
INTO    screening
VALUES (lower(hex(randomblob(16))), '18:30', 'Södran', '2025-03-03', 'tt0056058')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

-   Create a ticket where either the user or the performance doesn’t exist (this should fail).



In [18]:
%%sql
INSERT
INTO    tickets
VALUES ((lower(hex(randomblob(16)))), 'danne', '71872ace8b8c2964d3db89551ba14187')

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT
INTO    tickets
VALUES ((lower(hex(randomblob(16)))), 'danne', '71872ace8b8c2964d3db89551ba14187')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)